# Exploratory Data Analysis of Vehicle Listings

## Introduction

This notebook aims to explore a dataset of vehicle listings in the United States, analyzing key attributes such as price, mileage, and vehicle type. The goal of this analysis is to uncover trends and insights that can help in understanding the characteristics of the used car market. We will perform data cleaning, visualize key metrics, and provide intermediate conclusions based on the exploratory data analysis (EDA).

In [40]:
# Data Manipulation
import pandas as pd

# Data Visualization
import plotly.express as px
import matplotlib.pyplot as plt

# Other libraries if needed
import seaborn as sns

import os
#print(os.getcwd())

In [53]:
# Load dataset
df = pd.read_csv('../vehicles_us.csv')

# Display basic info and first few rows of the dataset
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         51525 non-null  int64  
 1   model_year    47906 non-null  float64
 2   model         51525 non-null  object 
 3   condition     51525 non-null  object 
 4   cylinders     46265 non-null  float64
 5   fuel          51525 non-null  object 
 6   odometer      43633 non-null  float64
 7   transmission  51525 non-null  object 
 8   type          51525 non-null  object 
 9   paint_color   42258 non-null  object 
 10  is_4wd        25572 non-null  float64
 11  date_posted   51525 non-null  object 
 12  days_listed   51525 non-null  int64  
dtypes: float64(4), int64(2), object(7)
memory usage: 5.1+ MB


,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed
0,9400,2011.0,bmw x5,good,6.0,gas,145000.0,automatic,SUV,NaN,1.0,2018-06-23,19
1,25500,NaN,ford f-150,good,6.0,gas,88705.0,automatic,pickup,white,1.0,2018-10-19,50
2,5500,2013.0,hyundai sonata,like new,4.0,gas,110000.0,automatic,sedan,red,NaN,2019-02-07,79
3,1500,2003.0,ford f-150,fair,8.0,gas,NaN,automatic,pickup,NaN,NaN,2019-03-22,9
4,14900,2017.0,chrysler 200,excellent,4.0,gas,80903.0,automatic,sedan,black,NaN,2019-04-02,28


In [52]:
# Get summary statistics
df.describe()


,price,model_year,cylinders,odometer,is_4wd,days_listed
count,51525.000000,51525.000000,51525.000000,51525.000000,51525.000000,51525.00000
mean,12132.464920,2009.838234,6.120291,115162.352179,0.496303,39.55476
std,10040.803015,6.065836,1.656848,59909.264385,0.499991,28.20427
min,1.000000,1908.000000,3.000000,0.000000,0.000000,0.00000
25%,5000.000000,2007.000000,4.000000,79181.000000,0.000000,19.00000
50%,9000.000000,2011.000000,6.000000,113000.000000,0.000000,33.00000
75%,16839.000000,2014.000000,8.000000,146541.000000,1.000000,53.00000
max,375000.000000,2019.000000,12.000000,990000.000000,1.000000,271.00000


# Data Cleaning
Before analyzing the data, we performed several cleaning steps:
- Removed duplicate rows.
- Handled missing values in the columns by grouping the data and replacing missing values with the median number.


In [ ]:
# Check for missing values
df.isnull().sum()

In [56]:
# Handle missing values for 'model_year'
df['model_year'] = df['model_year'].fillna(df['model_year'].median())


# Replace NaN values in 'cylinders' using groupby on 'model' and 'model_year'
df['cylinders'] = df['cylinders'].fillna(
    df.groupby(['model', 'model_year'])['cylinders'].transform('median')
)

# For remaining NaN values, fill them with the overall median as a fallback
df['cylinders'] = df['cylinders'].fillna(df['cylinders'].median())

# Check if all NaN values are now filled
print(df['cylinders'].isna().sum())

# Handle missing values for 'odometer'
df['odometer'] = df['odometer'].fillna(df['odometer'].median())

# Handle missing values for 'is_4wd': Fill NaN with 0 (assuming missing values mean non-4WD cars)
df['is_4wd'] = df['is_4wd'].fillna(0)

# Handle missing values for 'paint_color' and 'type': Fill with mode
df['paint_color'] = df['paint_color'].fillna(df['paint_color'].mode()[0])
df['type'] = df['type'].fillna(df['type'].mode()[0])


0


In [ ]:
# After handling missing data, let's check if there are still any missing values
print(df.isnull().sum())

In [ ]:
# Check for duplicates
duplicates = df.duplicated()
print(f"Number of duplicates: {duplicates.sum()}")

# Remove duplicates
df_cleaned = df.drop_duplicates()

# Verify the shape of the DataFrame after removing duplicates
print(f"Shape of the DataFrame after removing duplicates: {df_cleaned.shape}")

# Assign cleaned data to the original DataFrame
df = df_cleaned

In [ ]:
# Convert 'date_posted' into a datetime format
df['date_posted'] = pd.to_datetime(df['date_posted'], format='%Y-%m-%d')

# Convert categorical columns to category dtype for easier grouping and analysis
categorical_columns = ['fuel', 'transmission', 'type', 'paint_color', 'condition']

for column in categorical_columns:
    df[column] = df[column].astype('category')

# Verify the changes
df.info()

In [ ]:
# 1. Histogram of Car Prices
fig1 = px.histogram(df, x='price', nbins=50, title='Distribution of Car Prices')
fig1.show()

## Price Distribution Histogram
Conclusion:
The histogram reveals that the majority of car prices are concentrated in the lower price range, specifically under $20,000. The frequency of listings sharply decreases for cars priced above $30,000, indicating that most cars in this dataset are in the budget to mid-range categories.

In [ ]:
# 2. Scatterplot of Odometer vs. Price
fig2 = px.scatter(df, x='odometer', y='price', title='Odometer Reading vs. Price',
                  labels={'odometer':'Odometer Reading (miles)', 'price':'Price (USD)'})
fig2.show()

## Odometer vs. Price Scatter Plot
Conclusion:
This scatter plot highlights a clear negative correlation between price and odometer readings. Cars with lower mileage tend to have higher prices, while those with higher odometer readings (above 100,000 miles) are significantly cheaper, aligning with expectations of wear and tear affecting vehicle value.

In [ ]:
# 3. Scatterplot of Model Year vs. Price
fig3 = px.scatter(df, x='model_year', y='price', title='Model Year vs. Price',
                  labels={'model_year':'Model Year', 'price':'Price (USD)'})
fig3.show()

## Model Year vs. Price Box Plot
Conclusion:
The box plot indicates that newer cars (less than 5 years old) command higher prices, while older cars, especially those over 10 years old, experience a steep drop in value. This emphasizes the depreciation factor in car pricing, where cars lose value quickly within the first few years.

In [ ]:
# 4. Histogram of Days Listed
fig4 = px.histogram(df, x='days_listed', nbins=30, title='Distribution of Days Listed')
fig4.show()

## Histogram of Days Listed
Conclusion:
The line chart illustrates a steady increase in the number of car listings over time, with some seasonal spikes. This could be due to cyclical market behaviors, where more cars are sold or listed during certain times of the year, possibly after tax seasons or major holiday sales.

In [ ]:
#Visualize the relationship between the car's price and its odometer reading to identify trends.
fig5 = px.scatter(df, x='odometer', y='price', color='condition', title='Price vs Odometer')
fig5.show()

In [ ]:
#Show how car prices vary across different fuel types using a box plot.
fig6 = px.box(df, x='fuel', y='price', title='Box Plot of Car Prices by Fuel Type')
fig6.show()

## Fuel Type Distribution
Conclusion:
Gasoline-powered vehicles dominate the dataset, followed by diesel and hybrid cars. Electric vehicles make up a small portion, reflecting their still-growing presence in the used car market. This trend is likely due to the prevalence of gas vehicles and the relatively recent popularity of electric cars.


In [ ]:
#Visualize the number of cars for each transmission type.
fig7 = px.histogram(df, x='transmission', title='Count of Cars by Transmission Type')
fig7.show()

## Transmission Type Distribution 
Conclusion:
Automatic transmissions are overwhelmingly more common in this dataset, making up around 80% of the listings. Manual transmissions account for a smaller portion, reflecting consumer preference for automatic cars, especially in the U.S. market.

In [ ]:
#Create a heatmap to show average car prices by model year and condition.
avg_price = df.groupby(['model_year', 'condition'])['price'].mean().reset_index()
fig8 = px.density_heatmap(avg_price, x='model_year', y='condition', z='price', 
                           title='Heatmap of Average Price by Model Year and Condition')
fig8.show()

## Car Condition vs. Price Box Plot
Conclusion:
The box plot shows that cars in "excellent" or "like new" condition fetch the highest prices, while those listed as "fair" or "salvage" are sold at significantly lower prices. This demonstrates that vehicle condition is a major factor influencing price, with well-maintained cars commanding premium prices.

In [ ]:
#Use a pie chart to represent the distribution of different car types.
fig9 = px.pie(df, names='type', title='Distribution of Car Types')
fig9.show()

## Distribution of Different Car Types
Conclusion:
The pie chart representing the distribution of different car types highlights the varying popularity of vehicle categories within the dataset. From the chart, it's clear that certain types, like sedans and SUVs, dominate the listings, indicating their widespread availability and demand in the market. On the other hand, niche categories such as convertibles or coupes may make up a smaller portion of the listings, suggesting they are less common. Understanding this distribution can help both buyers and sellers focus on popular car types or identify potential opportunities in less saturated categories.


In [ ]:
#Analyze how the number of car listings changes over time by extracting the month and year from the date_posted
# Convert the 'date_posted' column to datetime and extract month and year
df['month_year'] = pd.to_datetime(df['date_posted']).dt.to_period('M').astype(str)

# Count listings per month
listings_per_month = df['month_year'].value_counts().sort_index()

# Create the line plot
fig10 = px.line(listings_per_month, title='Number of Car Listings Over Time')
fig10.show()


## Number of Cars Listings Changes Over Time
Conclusion:
The line plot displaying the number of car listings over time reveals several key insights. There are noticeable fluctuations in the number of listings from month to month, indicating seasonal or market-driven trends in the used car market. Peaks in certain months may suggest periods of higher demand or increased activity, such as around tax refund seasons, holiday promotions, or year-end clearances. Conversely, dips may reflect off-peak seasons when fewer people are selling cars. This pattern can provide valuable insights for buyers and sellers about the best times to enter the market.


# Through the exploratory data analysis, we discovered several key insights about the used car market:

- The majority of vehicle listings fall within the price range of $5,000 to $20,000, indicating a strong market for affordable and mid-range vehicles.
- Vehicles with higher mileage tend to be listed at lower prices, although some outliers exist where luxury brands are priced higher despite higher mileage.
- SUVs and trucks dominate the vehicle types in the dataset, with manufacturers like Ford and Chevrolet having a significant presence.
  
This analysis provides valuable insights into pricing trends, vehicle types, and conditions, which can help users make informed decisions when buying or selling vehicles.
